<h1> Efficien Net Learning and Understanding

In [16]:
import tensorflow as tf
from keras.applications.efficientnet import EfficientNetB0
from PIL import Image
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [17]:
model = EfficientNetB0()
# model.summary()
del model

In [18]:
def convert_image_to_tensor(path):
    #opens image from the given path and automatically comes in array format
    img = Image.open(path)
    # rescale to 224 x 224 size
    img = img.resize([224,224])
    #convert to numpy array
    img = np.asarray(img)
    return img


images_list = []
labels_list = []
labels_map = {}
label_val = 0
i = 'data_1/natural_images'
for folder in os.listdir(i):
    labels_map[label_val] = folder
    for file in os.listdir(i+'/'+folder):
        images_list.append(i+'/'+folder+'/'+file)
        labels_list.append(label_val)
    label_val +=1
labels_map

{0: 'dog',
 1: 'cat',
 2: 'car',
 3: 'person',
 4: 'motorbike',
 5: 'flower',
 6: 'airplane',
 7: 'fruit'}

In [19]:
print(len(images_list),len(labels_list))
layers = tf.keras.layers.CategoryEncoding( num_tokens= len(list(set(labels_list))),output_mode='one_hot')
labels_list = np.array(layers(labels_list))
images_list = np.array([convert_image_to_tensor(i) for i in images_list])

6899 6899


In [20]:
x,x_test,y,y_test = train_test_split(images_list,labels_list)

In [21]:
input_val = tf.keras.layers.Input(shape = (224,224,3))
model = EfficientNetB0(include_top=False, input_tensor=input_val,weights='imagenet')

for i in range(int(len(model.layers)* 0.75)) :
    model.layers[i].trainable = False

x = tf.keras.layers.GlobalAveragePooling2D(name = 'avg_pool')(model.output)
x = tf.keras.layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = tf.keras.layers.Dropout(top_dropout_rate,name = 'top_dropout')(x)

output = tf.keras.layers.Dense(len(labels_map),activation = 'softmax',name = 'pred')(x)

model = tf.keras.Model(inputs = input_val,outputs = output)
model.compile(
    optimizer = 'adam', loss = "categorical_crossentropy", metrics = ['accuracy']
)
# model.summary()

In [22]:
callback = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss',mode='auto',restore_best_weights=True )
hist=model.fit(x,y,validation_data=(x_test,y_test),epochs=3,callbacks=[callback],batch_size=128)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [24]:
def plot_metrics_for_training(history):

    plt.figure(figsize=(30, 15))
    plt.subplot(411)
    plt.plot(history['accuracy'], label='accuracy')
    plt.plot(history['val_accuracy'], label = 'val_accuracy')
    plt.xticks(np.arange(len(history['accuracy'])), np.arange(1, len(history['accuracy'])+1))
    #plt.xticks(np.arange(len(history['val_accuracy'])), np.arange(1, len(history['val_accuracy'])+1))
    plt.xlabel('Epoch')
    plt.ylabel('val_accuracy')
    plt.ylim([0.85, 1.05])
    plt.legend(loc='lower right')

    plt.subplot(412)
    plt.plot(history['loss'], label='loss')
    plt.plot(history['val_loss'], label = 'val_loss')
    plt.xticks(np.arange(len(history['loss'])), np.arange(1, len(history['loss'])+1))
    #plt.xticks(np.arange(len(history['val_accuracy'])), np.arange(1, len(history['val_accuracy'])+1))
    plt.xlabel('Epoch')
    plt.ylabel('val_loss')
    plt.ylim([0.001, 0.20])
    plt.legend(loc='lower right')

plot_metrics_for_training(hist.history)

NameError: name 'hist' is not defined